In [1]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [2]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.8.0
numpy: 1.21.5


In [3]:
# CONSTANTS:
MODEL_NAME = "STGCN"

In [122]:
from tensorflow.keras.activations import relu, softmax

class STGCN_Block(tf.keras.layers.Layer):
    
    def __init__(self, num_nodes, num_steps, num_features,
                       channels, kernel_size):
        super().__init__()
        
        self.num_nodes = num_nodes
        self.num_steps = num_steps
        self.num_features = num_features
        self.channels = channels
        
        self.tgc1 = tf.keras.layers.Conv1D(filters=2*channels[0],kernel_size=(kernel_size,),
                                           activation='linear')
        
        self.tgc2 = tf.keras.layers.Conv1D(filters=2*channels[2],kernel_size=(kernel_size,),
                                           activation='linear')
        
        self.gc = tf.keras.layers.Conv2D(filters=channels[0],kernel_size=(1, 1),
                                         activation='relu')
        
    def call(self,x,A):
        
        x = self.tgc1(x)
        x = tf.keras.activations.sigmoid(x[:,:,:,self.channels[0]:])*x[:,:,:,:self.channels[0]]
        
        x = tf.einsum('bnm,bmtd->bntd',A,x)
        x = self.gc(x)
        x = self.tgc2(x)
        x = tf.keras.activations.sigmoid(x[:,:,:,self.channels[2]:])*x[:,:,:,:self.channels[2]]
        
        return x


class STGCN(tf.keras.Model):
    
    def __init__(self, num_nodes, num_steps, num_features, out_features,
                       num_blocks, block_params, A=None, emb_dim=10):
        super().__init__()
        
        self.num_nodes = num_nodes
        self.num_steps = num_steps
        self.num_features = num_features
        self.num_blocks = num_blocks
        
        self.out_features = out_features
        
        self.adapt_adj = A is None
        
        if self.adapt_adj:
            
            self.time_mod1_sub = tf.keras.layers.Dense(emb_dim, activation='relu')
            self.time_mod2_sub = tf.keras.layers.Dense(emb_dim, activation='relu')
            
            self.time_mod1 = tf.keras.layers.Dense(emb_dim, activation='sigmoid')
            self.time_mod2 = tf.keras.layers.Dense(emb_dim, activation='sigmoid')
            
            self.E1 = tf.Variable(initial_value=tf.keras.initializers.RandomNormal(stddev=1)(shape=(num_nodes, emb_dim), 
                                                                                   dtype="float32"),trainable=True)
            
            self.E2 = tf.Variable(initial_value=tf.keras.initializers.RandomNormal(stddev=1)(shape=(num_nodes, emb_dim), 
                                                                                   dtype="float32"),trainable=True)
        else:
            self.A = A
            
        self.block_params = block_params
        
        self.block_layers = []
        
        for i in range(self.num_blocks):
            self.block_layers.append(STGCN_Block(num_nodes, num_steps, num_features,
                                                 channels=block_params[i][:-1], kernel_size=block_params[i][-1]))
        self.fc = tf.keras.layers.Dense(self.out_features*self.num_steps, activation='linear')
        self.reshaper = tf.keras.layers.Reshape((self.num_nodes, self.num_steps, self.out_features))
        
    
    def call(self,x):
        """
        Our input shape is (batch, nodes, time, features)
        """
        
        x = tf.transpose(x, perm=(0,2,1,3))
                
        if self.adapt_adj:
            time_input = x[:,:,:,-1]
            
            time_input1 = self.time_mod1_sub(time_input)
            time_input2 = self.time_mod2_sub(time_input)
            time_input1 = self.E1 * self.time_mod1(time_input1)
            time_input2 = self.E2 * self.time_mod2(time_input2)
            self.A = softmax(relu(tf.matmul(time_input1, time_input2, transpose_b=True)),axis=1)
        
        for i in range(self.num_blocks):
            x = self.block_layers[i](x, self.A)
        
        x = x[:,:,:1,:]
        x = self.fc(x)
        x = self.reshaper(x)
        x = tf.transpose(x, perm=(0,2,1,3))
        
        return x

In [131]:
num_nodes = input_shape[1]
num_steps = input_shape[2]

model = STGCN(num_nodes=207, num_steps=12, num_features=2, 
              out_features=2,num_blocks=2, block_params=[[64,32,64,3], [64,32,64,4]], 
              A=None, emb_dim=150)

In [132]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 100

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(1e-3))

model(scaled_data['x_train'][:32])
model.summary()

Model: "stgcn_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_269 (Dense)           multiple                  1950      
                                                                 
 dense_270 (Dense)           multiple                  1950      
                                                                 
 dense_271 (Dense)           multiple                  22650     
                                                                 
 dense_272 (Dense)           multiple                  22650     
                                                                 
 stgcn__block_114 (STGCN_Blo  multiple                 29760     
 ck)                                                             
                                                                 
 stgcn__block_115 (STGCN_Blo  multiple                 69952     
 ck)                                                      

In [133]:
scaled_data['x_train'].shape

(23974, 12, 207, 2)

In [134]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Epoch 1/100


2022-04-23 14:17:08.129387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - ETA: 0s - loss: 0.3208

2022-04-23 14:18:08.276316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


375/375 [==============================] - 64s 169ms/step - loss: 0.3208 - val_loss: 0.2783 - lr: 0.0010
Epoch 2/100
375/375 [==============================] - 62s 165ms/step - loss: 0.2546 - val_loss: 0.2463 - lr: 0.0010
Epoch 3/100
375/375 [==============================] - 62s 165ms/step - loss: 0.2324 - val_loss: 0.2281 - lr: 0.0010
Epoch 4/100
375/375 [==============================] - 61s 164ms/step - loss: 0.2222 - val_loss: 0.2218 - lr: 0.0010
Epoch 5/100
375/375 [==============================] - 61s 163ms/step - loss: 0.2174 - val_loss: 0.2213 - lr: 0.0010
Epoch 6/100
375/375 [==============================] - 61s 163ms/step - loss: 0.2134 - val_loss: 0.2124 - lr: 0.0010
Epoch 7/100
 64/375 [====>.........................] - ETA: 48s - loss: 0.2184

KeyboardInterrupt: 

In [21]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

2022-04-21 12:30:03.659151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))